In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [139]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**TEXT PROCESSING**

In [0]:
#get text input
path_to_file = "drive/My Drive/Colab Notebooks/targetdw.txt"
text = open(path_to_file, 'r').read() 

In [0]:
#get each unique character from the dataset
vocab = sorted(set(text)) 

In [0]:
#assign a number to each character & create two dictionaries that can go from numeric index to character and character to numeric index
char_to_ind = {char: ind for ind, char in enumerate(vocab)} 
ind_to_char = np.array(vocab) 

In [0]:
#encode all the text
encoded_txt = np.array([char_to_ind[c] for c in text]) 

**CREATING BATCHES**

In [0]:
seq_len = 200 

In [0]:
#get the total number of seq in input txt
total_num_seq = len(text) // (seq_len + 1) 

In [0]:
#creating training seq
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_txt) #convert txt vector into stream of character indices

In [0]:
sequences = char_dataset.batch(seq_len + 1, drop_remainder = True) 

In [0]:
#create input text seq and target txt seq which is shifted one character forward
def create_seq_targets(seq):
  input_txt = seq[:-1] #Hello my nam
  target_txt = seq[1:] #ello my name
  return input_txt, target_txt

In [0]:
#group them as a tuple
dataset = sequences.map(create_seq_targets) 

In [150]:
#test input text and target text by printing the first seq
for input_txt, target_txt in dataset.take(1):
  print (input_txt.numpy())
  print("".join(ind_to_char[input_txt.numpy()]))
  print('\n')
  print(target_txt.numpy())
  print("".join(ind_to_char[target_txt.numpy()]))

[14  0 49 62 67 67 58 71  1 46 54 64 58 72  1 27 65 65  0 95 29 61 58 56
 64  2 96  0 46 61 58 71 58  1 76 54 72  1 72 54 73 62 72 59 54 56 73 62
 68 67  1 62 67  1 46 74 71 65 68 74 60 61 96 72  1 75 68 62 56 58  1 54
 72  1 61 58  1 66 68 75 58 57  0 61 62 72  1 70 74 58 58 67  1 62 67 73
 68  1 69 68 72 62 73 62 68 67 11  1 34 58  1 61 54 57  1 61 62 72  1 68
 69 69 68 67 58 67 73  1 68 67  1 73 61 58  1 71 74 67  0 67 68 76  9  1
 54 67 57  1 75 58 71 78  1 72 68 68 67  1 73 61 58  1 76 61 62 73 58  1
 64 62 67 60  1 76 68 74 65 57  1 55 58  1 56 68 71 67 58 71 58 57  1 54
 67 57  0 56 68 66 69 65]
1
Winner Takes All
‘Check!’
There was satisfaction in Turlough’s voice as he moved
his queen into position. He had his opponent on the run
now, and very soon the white king would be cornered and
compl


[ 0 49 62 67 67 58 71  1 46 54 64 58 72  1 27 65 65  0 95 29 61 58 56 64
  2 96  0 46 61 58 71 58  1 76 54 72  1 72 54 73 62 72 59 54 56 73 62 68
 67  1 62 67  1 46 74 71 65 68 74 60 61 9

In [0]:
#generating training batches
batch_size = 64 
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder = True)

In [152]:
dataset

<BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int64, tf.int64)>

we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text


**CREATING THE MODEL**

In [0]:
vocab_size = len(vocab)

In [0]:
embed_dim = 74

In [0]:
rnn_neurons = 1024

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [0]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [0]:
#set up the loss function
def sparse_cat_loss(y_true, y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [0]:
#create the model
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [0]:
model = create_model(vocab_size=vocab_size,
                     embed_dim=embed_dim,
                     rnn_neurons=rnn_neurons,
                     batch_size=batch_size)

In [161]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (64, None, 74)            7474      
_________________________________________________________________
gru_8 (GRU)                  (64, None, 1024)          3379200   
_________________________________________________________________
dense_8 (Dense)              (64, None, 101)           103525    
Total params: 3,490,199
Trainable params: 3,490,199
Non-trainable params: 0
_________________________________________________________________


In [0]:
#generate txt with the model without training to check if it works
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [0]:
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [165]:
ind_to_char[sampled_indices]

array(['E', 'n', 'v', '‡', 'b', '5', 'a', '4', '3', 'l', '1', 'w', 'z',
       'x', 'X', 'J', 'S', 'I', '6', ',', 'æ', 'K', 'Q', 'j', '2', 'g',
       'l', 'u', '/', '(', 'w', 'â', '—', 'æ', 'l', '?', '1', '/', '-',
       't', '“', '‘', 'é', "'", 'g', '2', 'S', 'm', 'V', '%', 'K', 'J',
       'Q', 'B', ')', ':', 'l', 'k', 'é', 'A', 'Z', 'è', ':', 'T', 'Y',
       'X', '?', '0', 'œ', 'o', 'f', 'B', 'I', '-', 'â', '-', '(', '2',
       'Q', 'm', '7', 'J', '=', 'T', 'd', '-', '1', 'X', 'z', 'j', '£',
       'G', '7', 'E', 'e', 'o', 'I', 'F', '“', '’', 'P', 'o', 'æ', 'ë',
       'V', '!', ')', 'Z', 'L', 'a', 'd', 'v', '/', 'æ', ' ', '£', 'h',
       'v', 'p', 'J', 'n', 'J', '2', 'é', 't', 'r', ' ', '†', 'g', '-',
       'Y', '(', '”', '3', 'n', '/', '1', 'è', '"', '4', 'z', 'y', 't',
       'I', 'R', 'p', '–', 'H', '1', 'J', '8', 'x', 'O', 'h', 'z', 'w',
       '£', '\n', '“', 'M', 'Q', "'", 'm', 'l', 'â', 'V', 'k', 'C', 'i',
       'o', 'M', 'D', '3', '?', 'u', 'ë', '½', '!', '‘', 'T', '

**TRAINING THE MODEL**

In [0]:
epochs = 10

In [167]:
model.fit(dataset, epochs=epochs)

Epoch 1/10
1155/1155 [==============================] - 286s 247ms/step - loss: 1.8450
Epoch 2/10
1155/1155 [==============================] - 286s 248ms/step - loss: 1.2545
Epoch 3/10
1155/1155 [==============================] - 285s 247ms/step - loss: 1.1675
Epoch 4/10
1155/1155 [==============================] - 279s 241ms/step - loss: 1.1289
Epoch 5/10
1155/1155 [==============================] - 278s 241ms/step - loss: 1.1045
Epoch 6/10
1155/1155 [==============================] - 278s 240ms/step - loss: 1.0876
Epoch 7/10
1155/1155 [==============================] - 288s 249ms/step - loss: 1.0746
Epoch 8/10
1155/1155 [==============================] - 288s 249ms/step - loss: 1.0650
Epoch 9/10
1155/1155 [==============================] - 287s 249ms/step - loss: 1.0575
Epoch 10/10
1155/1155 [==============================] - 286s 248ms/step - loss: 1.0514


In [0]:
model.save('dw_model.h5')

**GENERATING TEXT**

In [0]:
from tensorflow.keras.models import load_model

In [0]:
#create model
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

#load trained weights
model.load_weights('dw_model.h5')

#build model with a batch_size of 1
model.build(tf.TensorShape([1, None]))

In [0]:
def generate_txt(model, start_seed,gen_size=100,temp=1.0):

  #number of characters to generate
  num_generate = gen_size

  #vecotrizing the word to create txt from
  input_eval = [char_to_ind[s] for s in start_seed]

  #expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  #empty list to hold resulting generated text
  txt_generated = []
 
  temperature = temp

  #clears hidden layer of the model
  model.reset_states()

  for i in range(num_generate):

      #generate predictions
      predictions = model(input_eval)

      #remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      #use a categorical disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      #pass the predicted character for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      #transform back to character letter
      txt_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(txt_generated))

In [185]:
print(generate_txt(model,"Doctor",gen_size=1000))

Doctor.
‘What? I dinned that Swordsman’s alcove rose out of the
castle. Herembirs the wounded outwards flowing dials.
Maren’s house was flat and so after they? Satisfied?T and
find light in a long and search.
Lowe lay somb roughly colonists’ spy... 
2
The Doctor’s habbed for
she had encountered. The Doctor’s eyes held bewildered.
Collins followed.
Two more questers on the edge of the Galaxt Was
the thought of settless happening, and saw that two set
wouldn’t lost – not in yearsh of an enormous tall man in
Boluce’ and she was hunched away—for the first couthing he could
see. Leela held out his shoulder and staggered on, ‘and
the man was planning to. Fasters on Earth some owive in there... Here, then it
would see now. You may have done so much fully like rather
more likely.’
‘I am in Solos and I got the signal affected our race.
Ran sprival over the planetary inhabitants, I seem—deceiving
maskem,’ snarled Gomer.
The conflict was the Doctor to tell me about Hildred. I
must go and find the